# Salarios - dev

#### Preprocesar datos no incluido en el dashboard final. Si se ha incluido el callback. 

[scatter](https://plotly.com/python/line-and-scatter/)

In [1]:
import pandas as pd
import plotly.express as px  
from dash import Dash, dcc, html, Input, Output
import numpy as np
import PyCurrency_Converter

#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv ('../data/survey_results_public2021.csv', index_col = [0]) 
# El indice sera la columna con el ID de la respuesta
df20 = pd.read_csv ('../data/survey_results_public2020.csv', index_col = [0])

# fuente: https://stats.bis.org/statx/srs/table/I3?c=&p=202204&m=A
df_currency = pd.read_csv ('../data/conversor.csv')

Se toman los datos del csv llamado conversor y se calcula la conversion del salario teniendo en cuenta su moneda y su frecuencia (salario mensual, semanal o anual).

In [2]:
# nombre columna: lo que han preguntado en el survey.
#currency: Which currency do you use day-to-day?
#comptotal: What is your current total compensation (answered in currency from currency column)
#compfreq: Is that compensation weekly, monthly, or yearly?

#copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
df=df21[['DevType','Currency', 'CompTotal', 'CompFreq']].dropna().copy()

df= df.drop(df[df.Currency == "none"].index) #eliminar todas las none currency

# 1. convertir todas las currency a la misma moneda. X ej dolares

#convertir la moneda a un string de 3 letras.
df['Currency']= df.Currency.str.split().str.get(0)

#unir ambos df 
df3 = pd.merge(df_currency, df, how='inner')

#sustituir la compensacion en la moneda del pais por su equivalente en USD
df3["CompTotal"] = (df3["CompTotal"]/df3["Conversion"]).round(2) #redondear el numero a 2 decmales

#ahora se debe normalizar la compensacion. Para ello se asumira que: 
# un mes tiene 4 seamanas y un anyo tiene 12 meses. 
# un anyo tiene tambien 52 semanas

condiciones = [(df3['CompFreq'] =='Monthly'), (df3['CompFreq'] =='Weekly'), (df3['CompFreq'] =='Yearly')]

df3['semana'] = np.select(condiciones, [(df3['CompTotal'])/4, df3['CompTotal'], (df3['CompTotal'])/52]).round(2)      
df3['mes'] = np.select(condiciones, [df3['CompTotal'], (df3['CompTotal'])*4, (df3['CompTotal'])/12]).round(2)      
df3['anyo'] = np.select(condiciones, [(df3['CompTotal'])*12, (df3['CompTotal'])*52, df3['CompTotal']]).round(2) 
df3

,Coutry,Currency,Conversion,DevType,CompTotal,CompFreq,semana,mes,anyo
0,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",820.17,Monthly,205.04,820.17,9842.04
1,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",84.62,Weekly,84.62,338.48,4400.24
2,Afghanistan,AFN,76.8135,"Developer, front-end;Developer, full-stack;Dev...",260.37,Monthly,65.09,260.37,3124.44
3,Afghanistan,AFN,76.8135,"Developer, mobile;Developer, front-end;Develop...",455.65,Monthly,113.91,455.65,5467.80
4,Afghanistan,AFN,76.8135,System administrator,13018.54,Monthly,3254.64,13018.54,156222.48
...,...,...,...,...,...,...,...,...,...
292977,Vietnam,VND,23152.9000,"Developer, mobile;Developer, desktop or enterp...",3455.29,Monthly,863.82,3455.29,41463.48
292978,Vietnam,VND,23152.9000,"Developer, back-end",993.40,Monthly,248.35,993.40,11920.80
292979,Yemen,YER,1240.7580,"Developer, mobile;Developer, front-end;Develop...",322.38,Monthly,80.60,322.38,3868.56
292980,Yemen,YER,1240.7580,"Developer, mobile;Developer, full-stack;Studen...",80.60,Monthly,20.15,80.60,967.20


# Rehacer la funcion. 

Hay una columna en el dataframe con la conversion hecha. Se conservara lo realizado hasta el momento.

## 1. Preprocesar datos.


In [3]:
def salario (df,anyo):
   
    if (anyo == 20): 
        #copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
        df4=df[['DevType','ConvertedComp', 'YearsCodePro','Age1stCode']].dropna().copy()
         #para poder reutilizar la columna en el df de 2021 y 2020 sin hacer grandes cambios, hay que renombrar ConvertedComp
        df4.rename(columns = {'ConvertedComp':'ConvertedCompYearly'}, inplace = True)
    
    if (anyo == 21): #Age1stCode en el df del 2021 son string en lugar de enteros.
         #copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
        df4=df[['DevType','ConvertedCompYearly', 'YearsCodePro','Age1stCode']].dropna().copy()
        
        #crear diccionario con los cambios de valores
        edadesTransformadas = {"Younger than 5 years" : 4, "5 - 10 years" : 7, "11 - 17 years": 18, "18 - 24 years" : 21,
        "25 - 34 years" : 30, "35 - 44 years" : 40, "45 - 54 years": 50, "55 - 64 years" : 60, 
        "Older than 64 years": 70}
        
        #cambiar los valores.
        df4=df4.replace({"Age1stCode": edadesTransformadas})
    
    #resto es igual para ambos df.
    
    # eliminar filas en anyos que tengan texto. 
    df4.drop(df4[df4['YearsCodePro'] == "Less than 1 year"].index, inplace = True) 
    df4.drop(df4[df4['YearsCodePro'] == "More than 50 years"].index, inplace = True)
    
    df4.drop(df4[df4['Age1stCode'] == "Younger than 5 years"].index, inplace = True) 
    df4.drop(df4[df4['Age1stCode'] == "Older than 85"].index, inplace = True)
    
    #convertir columnas a entero
    df4["Age1stCode"] = pd.to_numeric(df4["Age1stCode"])
    df4["YearsCodePro"] = pd.to_numeric(df4["YearsCodePro"])
    
    #separar filas con mas de un devtype y despues agruparlas
    df4=df4.assign(DevType=df4['DevType'].str.split(';')).explode('DevType').groupby('DevType').agg(avg_pro=('YearsCodePro', 'mean'), #nueva columna que calculara la media de yearscodepro
                 avg_age=('Age1stCode','mean'), 
                 median_money=('ConvertedCompYearly','median'), #nueva columna con mediana del salario
                 # contar los diferentes tipos de devtype                                                                                    
                 respuestas=('DevType','count')).reset_index() #reiniciar el indice

    #eliminar informacion irrelevante
    df4.drop(df4.index[df4['DevType'] == "Other (please specify):"], inplace=True)
    
    return df4

## 2. Layout. 


In [4]:
app = Dash(__name__)
server = app.server #heroku
app.layout = html.Div([

    html.H1("Tipo de desarrollador", style={'text-align': 'center'}), #cabecero h1. Header
    
    #primera mini prueba con un menu desplegable.
    dcc.Dropdown(id="opt4",  
                 options=[ #el usuario va a ver las label.
                     {"label": "2020", "value": 2020},
                     {"label": "2021", "value": 2021}],
                 multi=False,
                 value=2020,
                 style={'width': "40%"}
                 ),
    
    html.Button('Button 1', id='hide-show', n_clicks=0), #ocultar o enseñar leyenda

    dcc.Graph(id='my_survey', figure={}) # graph container

])

# 3. callback

En este caso, un [scatter](https://plotly.com/python/line-and-scatter/).

In [5]:
@app.callback(
    [Output(component_id='my_survey', component_property='figure'),
    Output('hide-show', 'children')],
    [Input(component_id='opt4', component_property='value'),
    Input(component_id='hide-show', component_property='n_clicks')])
def update_graph(opt4, n_clicks): #se pasan los dos input.
    
    df = pd.DataFrame()
    anyo = 0

    if (opt4 == 2020): 
        df= df20 
        anyo = 20
    else: 
        df = df21 
        anyo = 21

    fig =  px.scatter(salario (df,anyo), x= "avg_pro", y= "avg_age", size="respuestas", 
                      color="DevType",  hover_name="DevType", text = "DevType", hover_data=["median_money"],
                      labels={'avg_pro':'Years Coding Profesionaly (average)',
                              'avg_age':'Years since start coding (average)',
                              'respuestas':'# responses','median_money':'Median salary ($)', 
                              'DevType':'Dev type'},
                     color_discrete_sequence=px.colors.qualitative.Pastel,)
    
    fig.update_traces(textposition='top center') # texto que va encima de los puntitos

    #quitar color y grid del grafico
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
    
    # se introduce por primera vez un boton que pone o quita la leyenda.
    bool_disabled = n_clicks % 2
    if bool_disabled:
        return fig, "Hide Leyend" # se devuelven los dos output
    else:
        fig.update_layout(showlegend=False)
        return fig, "Show Leyend"

## 4. run server

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
